In [31]:
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))

In [32]:
import torch

from src.model import VAE
from src.dataset import CustomDataset
from torch.utils.data import DataLoader
from torchvision import transforms

In [33]:
train_transform = transforms.Compose([
	transforms.Resize((256, 256)),
	transforms.RandomCrop(224),
	transforms.RandomHorizontalFlip(0.5),
	transforms.ToTensor(),
	transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])
test_transform = transforms.Compose([
	transforms.Resize((256, 256)),
	transforms.ToTensor(),
])

train_data = CustomDataset(
	root='../data/CelebA/',
	split='train',
	transform=train_transform
)
test_data = CustomDataset(
	root='../data/CelebA/',
	split='test',
	transform=test_transform
)

train_loader = DataLoader(
	dataset=train_data,
	batch_size=128,
	shuffle=True,
	drop_last=True
)
test_loader = DataLoader(
	dataset=test_data,
	batch_size=128,
	shuffle=False,
	drop_last=False
)

In [34]:
def train(train_loader, learning_rate=0.0001, epochs=10):
	device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
	device = "cpu"
	print("Using {} device".format(device))

	model = VAE().to(device)
	optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
	criterion = model.loss_function

	for epoch in range(epochs):
		for i, data in enumerate(train_loader):
			data = data.to(device)
			optimizer.zero_grad()
			outputs = model(data)
			result = criterion(*outputs, M_N=0.005)
			loss = result['loss']
			recon_loss = result['Reconstruction_Loss']
			kl_loss = result['KLD']
			loss.backward()
			optimizer.step()
			#if i % 100 == 0:
			print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Recon Loss: {:.4f}, KL Div: {:.4f}'.format(
				epoch + 1, epochs, i + 1, len(train_loader), loss.item(), recon_loss.item(), kl_loss.item()
			))
	
	return model

In [35]:
model = train(train_loader)

Using cpu device
Epoch [1/10], Step [1/949], Loss: 35232008.0000, Recon Loss: 35232008.0000, KL Div: -146.7695
Epoch [1/10], Step [2/949], Loss: nan, Recon Loss: nan, KL Div: -inf
Epoch [1/10], Step [3/949], Loss: nan, Recon Loss: nan, KL Div: nan


[E thread_pool.cpp:110] Exception in thread pool task: mutex lock failed: Invalid argument
[E thread_pool.cpp:110] Exception in thread pool task: mutex lock failed: Invalid argument
[E thread_pool.cpp:110] Exception in thread pool task: mutex lock failed: Invalid argument
[E thread_pool.cpp:110] Exception in thread pool task: mutex lock failed: Invalid argument


KeyboardInterrupt: 